In [1]:
# Model Selection
# ===============
# This notebook compares the scores of different model algorithms, 
# preprocessing methods, and classification methods.
#
# Copyright 2020, 2021 Jerrad M. Genson
#
# This Source Code Form is subject to the terms of the Mozilla Public
# License, v. 2.0. If a copy of the MPL was not distributed with this
# file, You can obtain one at https://mozilla.org/MPL/2.0/.

library(tidyverse)

GIT_ROOT <- system2('git', args=c('rev-parse', '--show-toplevel'), stdout=TRUE)
DATA <- file.path(GIT_ROOT, 'data')
MODEL_DATA <- file.path(DATA, 'model_selection.csv')

scores <- read_csv(MODEL_DATA)
scores

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  model = col_character(),
  preprocessing = col_cha

model,preprocessing,accuracy,precision,sensitivity,specificity,informedness,dor,ami,outlier informedness,cv informedness,mad informedness,commit hash
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
qda,robust scaling,0.8140,0.8095,0.8763,0.7333,0.6096,19.480,0.2993,0.4833,0.6036,0.08135,5236d5b
lda,robust scaling,0.8140,0.8155,0.8660,0.7467,0.6126,19.040,0.2981,0.5500,0.5782,0.07821,9b44485
dtc,robust scaling,0.7791,0.7565,0.8969,0.6267,0.5236,14.600,0.2389,0.6500,0.5143,0.07692,ad170e6
sgb,robust scaling,0.8081,0.7963,0.8866,0.7067,0.5933,18.830,0.2895,0.5667,0.5879,0.07756,87c9f52
rfc,robust scaling,0.8314,0.8269,0.8866,0.7600,0.6466,24.760,0.3384,0.5500,0.5627,0.08640,604f618
rfc,isomap,0.8081,0.8200,0.8454,0.7600,0.6054,17.310,0.2851,0.4667,0.6357,0.11830,614ff30
rfc,pca,0.8430,0.8365,0.8969,0.7733,0.6702,29.680,0.3668,0.6500,0.5782,0.09998,e16ba46
rfc,lle,0.7965,0.8444,0.7835,0.8133,0.5968,15.770,0.2693,0.4667,0.5334,0.14640,4bdf43f
rfc,lle hessian,0.8198,0.8235,0.8660,0.7600,0.6260,20.460,0.3105,0.4833,0.5205,0.09570,3be6109


In [2]:
best_informedness <- scores[scores$informedness == max(scores$informedness), ]
best_informedness

model,preprocessing,accuracy,precision,sensitivity,specificity,informedness,dor,ami,outlier informedness,cv informedness,mad informedness,commit hash
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
knn,factor analysis,0.8721,0.8788,0.8969,0.84,0.7369,45.67,0.4408,0.6333,0.6164,0.1071,05e50d5


In [3]:
best_dor <- scores[scores$dor == max(scores$dor), ]
best_dor

model,preprocessing,accuracy,precision,sensitivity,specificity,informedness,dor,ami,outlier informedness,cv informedness,mad informedness,commit hash
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
knn,factor analysis,0.8721,0.8788,0.8969,0.84,0.7369,45.67,0.4408,0.6333,0.6164,0.1071,05e50d5


In [4]:
best_sensitivity <- scores[scores$sensitivity == max(scores$sensitivity), ]
best_sensitivity

model,preprocessing,accuracy,precision,sensitivity,specificity,informedness,dor,ami,outlier informedness,cv informedness,mad informedness,commit hash
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
dtc,robust scaling,0.7791,0.7565,0.8969,0.6267,0.5236,14.60,0.2389,0.6500,0.5143,0.07692,ad170e6
rfc,pca,0.8430,0.8365,0.8969,0.7733,0.6702,29.68,0.3668,0.6500,0.5782,0.09998,e16ba46
knn,pca,0.8372,0.8286,0.8969,0.7600,0.6569,27.55,0.3535,0.7500,0.5506,0.12090,7ace118
knn,factor analysis,0.8721,0.8788,0.8969,0.8400,0.7369,45.67,0.4408,0.6333,0.6164,0.10710,05e50d5


In [5]:
best_cv_informedness <- scores[scores$`cv informedness` == max(scores$`cv informedness`), ]
best_cv_informedness

model,preprocessing,accuracy,precision,sensitivity,specificity,informedness,dor,ami,outlier informedness,cv informedness,mad informedness,commit hash
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
rfc,isomap,0.8081,0.82,0.8454,0.76,0.6054,17.31,0.2851,0.4667,0.6357,0.1183,614ff30


In [6]:
second_best_informedness <- scores[scores$informedness == unique(sort(scores$informedness, TRUE))[2], ]
second_best_informedness

model,preprocessing,accuracy,precision,sensitivity,specificity,informedness,dor,ami,outlier informedness,cv informedness,mad informedness,commit hash
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
lrc,lle,0.8547,0.875,0.866,0.84,0.706,33.92,0.3948,0.7333,0.541,0.1554,0330f84


In [7]:
third_best_informedness <- scores[scores$informedness == unique(sort(scores$informedness, TRUE))[3], ]
third_best_informedness

model,preprocessing,accuracy,precision,sensitivity,specificity,informedness,dor,ami,outlier informedness,cv informedness,mad informedness,commit hash
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
knn,robust scaling,0.8372,0.8791,0.8247,0.8533,0.6781,27.38,0.3575,0.55,0.6219,0.1052,3ee0a51
knn,feature agglomeration,0.8372,0.8791,0.8247,0.8533,0.6781,27.38,0.3575,0.55,0.6219,0.1052,10fa6ee
